In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


In [7]:
# Load dataset
file_path = r"C:\Users\user\Desktop\Task_Timing_Log_1000_Entries.xlsx"
df = pd.read_excel(file_path)


In [9]:
# Preprocess
df['Start Time'] = pd.to_datetime(df['Start Time'])
df['End Time'] = pd.to_datetime(df['End Time'])
df['Duration'] = (df['End Time'] - df['Start Time']).dt.total_seconds() / 60.0
df['Task Name'] = df['Task Name'].str.strip().str.lower()
df = df.sort_values(by=['Task Name', 'Start Time'])
df['Repetition'] = df.groupby('Task Name').cumcount() + 1


In [11]:
# Map task name to ID
task_to_id = {task: i for i, task in enumerate(df['Task Name'].unique())}
df['Task ID'] = df['Task Name'].map(task_to_id)

In [13]:
# Feature matrix
X = df[['Repetition', 'Duration', 'Task ID']].values.astype(np.float32)
y = df['Quality Score'].values.astype(np.float32).reshape(-1, 1)


In [15]:
# Torch tensors
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)



In [17]:
# Neural Network
class LearningModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 12)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(12, 8)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(8, 1)
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)



In [19]:
model = LearningModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [21]:
# Simulate Synaptic Fatigue (diminishing returns)
def fatigue_penalty(repetition):
    return 1 / (1 + 0.05 * (repetition - 1) ** 2)


In [23]:
# Training Loop
losses = []
for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_tensor)
    penalties = torch.tensor(fatigue_penalty(X[:, 0]), dtype=torch.float32).reshape(-1, 1)
    penalized_targets = y_tensor * penalties
    loss = criterion(outputs, penalized_targets)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    if epoch % 50 == 0:
        print(f"Epoch {epoch} Loss: {loss.item():.4f}")



Epoch 0 Loss: 19.0533
Epoch 50 Loss: 0.7294
Epoch 100 Loss: 0.4918
Epoch 150 Loss: 0.3490
Epoch 200 Loss: 0.2935
Epoch 250 Loss: 0.2616


In [25]:
# Predict
model.eval()

# ✅ User Input Prediction Loop
while True:
    print("\n🔍 Predict Quality Score:")
    task_name = input("Enter task name (or 'exit' to quit): ").strip().lower()
    if task_name == 'exit':
        break

    if task_name not in task_to_id:
        print("❌ Unknown task name. Please try one from the dataset.")
        continue

    try:
        repetition = int(input("Enter repetition count (e.g., 3): "))
        duration = float(input("Enter duration in minutes (e.g., 90): "))
    except ValueError:
        print("❌ Invalid input. Please enter numeric values for repetition and duration.")
        continue

    task_id = task_to_id[task_name]
    input_tensor = torch.tensor([[repetition, duration, float(task_id)]], dtype=torch.float32)
    predicted_score = model(input_tensor).item()

    print(f"📘 Predicted Quality Score for '{task_name.title()}' (Attempt {repetition}, {duration:.1f} mins): {predicted_score:.2f}")



🔍 Predict Quality Score:


Enter task name (or 'exit' to quit):  Study Math
Enter repetition count (e.g., 3):  3
Enter duration in minutes (e.g., 90):  90


📘 Predicted Quality Score for 'Study Math' (Attempt 3, 90.0 mins): 5.88

🔍 Predict Quality Score:


Enter task name (or 'exit' to quit):  Revise Notes
Enter repetition count (e.g., 3):  2
Enter duration in minutes (e.g., 90):  15


📘 Predicted Quality Score for 'Revise Notes' (Attempt 2, 15.0 mins): 3.70

🔍 Predict Quality Score:


Enter task name (or 'exit' to quit):  exit
